In [ ]:
!pip install transformers datasets accelerate evaluate

# ***Create Data***

In [10]:
from datasets import load_dataset

# Load tweet_eval sentiment dataset
dataset = load_dataset("tweet_eval", "sentiment")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


# ***Load tokenizer***

In [11]:
from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# ***Tokenize the dataset***

In [12]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128          # Shorter tweets
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/45615 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# ***Load model***

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # tweet_eval sentiment: 0-negative,1-neutral,2-positive
    id2label={0: "negative", 1: "neutral", 2: "positive"},
    label2id={"negative": 0, "neutral": 1, "positive": 2}
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# ***metrics (accuracy)***

In [16]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# ***Prepare training arguments***

In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    logging_steps=10,
    report_to="none",
    fp16=True,  # Mixed precision if GPU available
)

# ***Trainer***

In [19]:
from transformers import Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-1745581428.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()

Step,Training Loss
10,1.078800
20,1.019700
30,1.000700
40,1.013100
50,1.004600
60,0.935900
70,0.909100
80,0.840200
90,0.832700
100,0.861200


TrainOutput(global_step=8553, training_loss=0.515618148553055, metrics={'train_runtime': 1114.9198, 'train_samples_per_second': 122.74, 'train_steps_per_second': 7.671, 'total_flos': 4531956111832320.0, 'train_loss': 0.515618148553055, 'epoch': 3.0})

# ***evaluate***

In [21]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6967777013778687, 'eval_accuracy': 0.7405, 'eval_runtime': 2.1614, 'eval_samples_per_second': 925.315, 'eval_steps_per_second': 57.832, 'epoch': 3.0}


# ***Saving***

In [22]:
trainer.save_model("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

('./sentiment_model/tokenizer_config.json',
 './sentiment_model/special_tokens_map.json',
 './sentiment_model/vocab.txt',
 './sentiment_model/added_tokens.json',
 './sentiment_model/tokenizer.json')

# ***Prediction***

In [24]:
from transformers import pipeline
import torch

def test_classifier():
    # Use GPU if available
    device = 0 if torch.cuda.is_available() else -1

    # Load the fine-tuned sentiment classifier
    classifier = pipeline(
        "text-classification",  # Correct pipeline type
        model="./sentiment_model",
        tokenizer="./sentiment_model",
        device=device
    )

    # Example test tweets
    test_reviews = [
        "This is the best product I've ever used!",
        "Absolutely terrible, waste of money.",
        "It's okay, nothing special.",
        "Amazing quality and fast shipping!",
        "Disappointed with the purchase.",
    ]

    print("\n" + "="*60)
    print("Testing Tweet Classifier")
    print("="*60 + "\n")

    # Run classifier on each review
    for review in test_reviews:
        result = classifier(review)[0]  # Returns a dict with 'label' and 'score'
        print(f"Review: {review}")
        print(f"→ Sentiment: {result['label']} (confidence: {result['score']:.2%})")
        print("-" * 60)

if __name__ == "__main__":
    test_classifier()

Device set to use cuda:0



Testing Tweet Classifier

Review: This is the best product I've ever used!
→ Sentiment: positive (confidence: 99.43%)
------------------------------------------------------------
Review: Absolutely terrible, waste of money.
→ Sentiment: negative (confidence: 98.84%)
------------------------------------------------------------
Review: It's okay, nothing special.
→ Sentiment: positive (confidence: 69.62%)
------------------------------------------------------------
Review: Amazing quality and fast shipping!
→ Sentiment: positive (confidence: 99.53%)
------------------------------------------------------------
Review: Disappointed with the purchase.
→ Sentiment: negative (confidence: 98.62%)
------------------------------------------------------------
